In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
from os.path import join, isfile
from os import listdir
from functools import partial
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('ticks')
sns.set_context('paper')

import glam as gb

%load_ext autoreload
%autoreload 2

# Example 3: Parameter recovery

When performing model-based analyses of behaviour that include the interpretation of parameter estimates, or comparisons of parameter estimates between groups or conditions, the researcher should be confident that the model’s parameters are actually identifiable. In particular, the researcher needs to be confident, that the set of estimated parameters unambiguously describes the observed data better than any other set of parameters. A straightforward way of testing this is to perform a parameter recovery: The general intuition of a parameter recovery analysis is to first generate a synthetic data set from a model using a set of known parameters, and then fitting the model to the synthetic data. Finally, the estimated parameters can be compared to the known generating parameters. If they match to a satisfying degree, the parameters were recovered successfully. Previous analyses have already indicated that the GLAM’s parameters can generally be recovered to a satisfying degree (see Thomas et al., 2019). Yet, the ability to identify a specific set of parameters always depends on the specific features of a given dataset. The most obvious feature of a data set that influences recoverability of model parameters is the number of data points included. Usually this quantity refers to the number of trials. For hierarchical models, the precision of group level estimates also depends on the number of individuals per group. Additional features that vary between datasets and that could influence parameter estimation are the observed distribution of gaze, the distribution of item values or the number of items in each trial. For this reason, it should always be tested whether the estimated parameters of a model can be recovered in the context of a specific dataset.

In [2]:
# Simulate a pretend-to-be-collected dataset.
# In the real world, this dataset is collected from participants, so we do not know
# 1) if GLAM is an adequate model for the data
# 2) data-generating parameters

data_model = gb.GLAM()

parameters = dict(v=dict(mu=0.000075, sd=0.000025, bounds=(0.00002, 0.0002)),
                  gamma=dict(mu=0.1, sd=0.3, bounds=(-1, 1)),
                  s=dict(mu=0.008, sd=0.001, bounds=(0.005, 0.015)),
                  tau=dict(mu=0.8, sd=0.1, bounds=(0.1, 2)))

data_model.simulate_group(kind='hierarchical',
                          n_individuals=50,
                          n_trials=200,
                          n_items=3,
                          parameters=parameters,
                          value_range=(1, 10),
                          seed=1)

data = data_model.data

In [3]:
data.to_csv(join('examples', 'example_3', 'data', 'data.csv'), index=False)

## Example Recovery

To demonstrate the procedure of a basic parameter recovery analysis using GLAMbox, suppose we have collected and loaded a data set called data. In the first step, we perform basic parameter estimation as in the previous examples:

In [4]:
glam = gb.GLAM(data=data)
glam.make_model(kind='individual')
glam.fit(method='MCMC', draws=5000, tune=5000, chains=4)

Generating single subject models for 50 subjects...


/Users/felixmolter/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/felixmolter/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/felixmolter/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tupl

Fitting 50 model(s) using MCMC...
  Fitting model 1 of 50...


/Users/felixmolter/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/felixmolter/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/felixmolter/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tupl

  Fitting model 2 of 50...


/Users/felixmolter/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1580.91draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 3 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1586.85draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 4 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1570.98draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 5 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1189.73draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 6 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1577.54draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 7 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1571.34draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 8 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1565.04draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 9 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:26<00:00, 1526.72draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 10 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1563.61draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 11 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1548.95draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 12 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1574.90draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 13 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1568.20draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 14 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1563.22draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 15 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1565.64draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 16 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1556.59draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 17 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1563.01draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 18 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1556.36draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 19 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1570.00draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 20 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1560.14draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 21 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1563.95draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 22 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1566.20draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 23 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1561.66draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 24 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1552.91draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 25 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1560.58draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 26 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1556.35draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 27 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1566.55draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 28 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1553.59draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 29 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1557.54draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 30 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1563.79draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 31 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1568.28draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 32 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1554.32draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 33 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1569.20draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 34 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1570.34draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 35 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1561.12draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 36 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1553.34draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 37 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1558.02draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 38 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1563.34draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 39 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1542.10draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 40 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1560.83draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 41 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1553.32draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 42 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1553.20draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 43 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1560.93draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 44 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1548.25draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 45 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1560.16draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 46 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1553.44draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 47 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1560.74draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 48 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1551.70draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 49 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1545.05draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 50 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1559.15draws/s]
The number of effective samples is smaller than 10% for some parameters.


/!\ Automatically setting parameter precision...


The next step is to create a synthetic, model-generated data set using the model parameters estimated from the empirical data, together with the empirically observed stimulus and gaze data. Setting`n_repeats` to 1 results in a data set of the same size as the observed one:

In [5]:
glam.predict(n_repeats=1)
synthetic = glam.prediction

  0%|          | 9/10000 [00:00<02:00, 83.09it/s]

Generating predictions for 10000 trials (1 repeats each)...


100%|██████████| 10000/10000 [02:02<00:00, 81.63it/s]


In [6]:
synthetic.to_csv(join('examples', 'example_3', 'data', 'synthetic.csv'), index=False)

In [16]:
# For this synthetic dataset, we know the generating parameters:
true_parameters = {parameter: glam.estimates[parameter].values
                   for parameter in ['v', 'gamma', 's', 'tau']}

# Save these generating parameters
true_param_df = pd.DataFrame(true_parameters)
true_param_df['subject'] = range(50)
true_param_df.to_csv(join('examples', 'example_3', 'results', 'true_parameters.csv'), index=False)

This data set should resemble the empirically observed data closely. Next, we create a new model instance, attach the synthetic data, build a hierarchical model and re-estimate its parameters:

/Users/felixmolter/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/felixmolter/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [8]:
gb.plots.plot_aggregate(data, line_data=[synthetic], line_labels=['Synthetic Data'])
plt.savefig(join('examples', 'example_3', 'figures', 'ex3_fig1_data-vs-synthetic.png'), dpi=300)

In [9]:
glam_rec = gb.GLAM(data=synthetic)
glam_rec.make_model(kind='individual')
glam_rec.fit(method='MCMC', draws=5000, tune=5000, chains=4)

Generating single subject models for 50 subjects...


/Users/felixmolter/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


Fitting 50 model(s) using MCMC...
  Fitting model 1 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1562.40draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 2 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1564.16draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 3 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1570.05draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 4 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1560.68draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 5 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1557.15draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 6 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1568.93draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 7 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1560.10draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 8 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1559.56draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 9 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1553.59draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 10 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1554.78draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 11 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1548.41draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 12 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1560.30draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 13 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1539.49draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 14 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1555.80draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 15 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1553.58draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 16 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1552.41draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 17 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1560.88draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 18 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1554.76draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 19 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1557.87draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 20 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1556.20draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 21 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:26<00:00, 1529.04draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 22 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1548.81draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 23 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1554.13draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 24 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1549.21draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 25 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1546.07draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 26 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1544.18draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 27 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1546.92draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 28 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1562.09draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 29 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1544.34draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 30 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1542.93draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 31 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1552.53draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 32 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1550.36draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 33 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1555.98draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 34 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1550.97draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 35 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1561.04draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 36 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1557.21draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 37 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1564.61draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 38 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1547.44draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 39 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1546.88draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 40 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1551.47draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 41 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1553.82draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 42 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1552.21draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 43 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1542.14draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 44 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1545.39draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 45 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1552.48draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 46 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1539.47draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 47 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1545.82draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 48 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1552.83draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 49 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1555.13draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 50 of 50...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 4 chains: 100%|██████████| 40000/40000 [00:25<00:00, 1557.66draws/s]
The number of effective samples is smaller than 10% for some parameters.


/!\ Automatically setting parameter precision...


In [17]:
glam_rec.estimates

,gamma,gamma_hpd_2.5,gamma_hpd_97.5,s,s_hpd_2.5,s_hpd_97.5,subject,t0,t0_hpd_2.5,t0_hpd_97.5,tau,tau_hpd_2.5,tau_hpd_97.5,v,v_hpd_2.5,v_hpd_97.5
0,0.05,-0.018554,0.113032,0.008589,0.007671,0.009459,0,0.0,0.0,0.0,0.82,0.600884,1.076278,0.000122,0.000113,0.000130
1,0.42,0.334203,0.561814,0.007327,0.006656,0.008101,1,0.0,0.0,0.0,0.39,0.297999,0.551529,0.000065,0.000060,0.000070
2,0.27,0.199903,0.314229,0.006442,0.005812,0.007198,2,0.0,0.0,0.0,1.02,0.782735,1.535478,0.000060,0.000056,0.000065
3,0.28,0.201041,0.375621,0.006903,0.006197,0.007601,3,0.0,0.0,0.0,0.75,0.555189,1.008060,0.000049,0.000045,0.000052
4,-0.18,-0.255495,-0.106678,0.008729,0.007708,0.009678,4,0.0,0.0,0.0,0.95,0.636418,1.465469,0.000092,0.000084,0.000101
5,-0.01,-0.233836,0.251144,0.008259,0.007427,0.008990,5,0.0,0.0,0.0,0.48,0.311331,0.845348,0.000023,0.000019,0.000027
6,0.23,0.180262,0.302711,0.009021,0.008001,0.009791,6,0.0,0.0,0.0,0.87,0.615516,1.127702,0.000121,0.000113,0.000129
7,-0.13,-0.241143,-0.067177,0.008667,0.007953,0.009535,7,0.0,0.0,0.0,1.12,0.819345,1.619605,0.000056,0.000051,0.000061
8,0.03,-0.033451,0.117763,0.007604,0.007076,0.008567,8,0.0,0.0,0.0,0.73,0.591427,0.948087,0.000081,0.000076,0.000087
9,0.19,0.121217,0.241956,0.008072,0.007218,0.008888,9,0.0,0.0,0.0,0.76,0.594150,1.236636,0.000067,0.000062,0.000072


Finally, the recovered and generating parameters can be compared. If the recovered parameters do not align with the generating parameters, this indicates that model parameters cannot be identified given this specific data set. In this case, parameter estimates should not be interpreted, as the true parameters might be different from the obtained estimates.
If, on the other hand, generating and recovered parameters do align to a satisfying degree, the parameters have been recovered successfully. This increases confidence in that the parameters obtained from the empirical data are valid and can be interpreted.

In [10]:
def plot_recovery_individual(model, generating_parameters,
                             parameters=['v', 'gamma', 's', 'tau'],
                             xlimits=dict(v=[-0.00003, 0.00003],
                                          gamma=[-0.25, 0.25],
                                          s=[-0.0025, 0.0025],
                                          tau=[-1, 1]),
                             figsize=(12, 4)):
    """
    Plot parameter recovery results from individually fitted models.
    
    Args:
        model: Fitted GLAM model of type 'individual'
        generating_parameters (dict): Dictionary of data generating parameters
        parameters (list, optional): List of parameters to include
        figsize (tuple, optional): Figure size
    
    
    """
    parameter_names = {'v': 'v',
                       'gamma': r'$\gamma$',
                       's': r'$\sigma$',
                       'tau': r'$\tau$'}

    n_individuals = len(generating_parameters[parameters[0]])
    n_parameters = len(parameters)

    # Construct long dataframe, 
    # every row is one parameter of one subject
    recovery = []
    for parameter in parameters:
        recovery_p = model.estimates[['subject', parameter, parameter + '_hpd_2.5', parameter + '_hpd_97.5']].copy()
        recovery_p.rename({parameter: 'recovered',
                           parameter + '_hpd_2.5': 'recovered_hpd_lower',
                           parameter + '_hpd_97.5': 'recovered_hpd_upper'},
                          axis=1, inplace=True)
        recovery_p['parameter'] = parameter
        recovery_p['generating'] = generating_parameters[parameter]
        recovery.append(recovery_p)
    recovery = pd.concat(recovery)
    recovery['success'] = ((recovery['generating'] > recovery['recovered_hpd_lower']) &
                           (recovery['generating'] < recovery['recovered_hpd_upper'])).values

    # Plot
    fig, axs = plt.subplots(1, n_parameters, figsize=figsize)

    for parameter, ax in zip(parameters, axs):
        parameter_df = recovery.loc[recovery['parameter'] == parameter]
    
        # Success Color coding
        color = np.array(['red', 'green'])[parameter_df['success'].values.astype(int)]
        
        # Vertical, indicating zero difference
        ax.axvline(0, color='black', zorder=-1, linewidth=1, alpha=0.7)
        # Difference posterior mean - generating
        ax.scatter(x=(parameter_df['recovered'] - parameter_df['generating']).values,
                   y=range(n_individuals),
                   color=color,
                   marker='o', facecolor='white',
                   zorder=2)
        # HPD        
        ax.hlines(y=range(n_individuals),
                  xmin=parameter_df['recovered_hpd_lower'].values - parameter_df['generating'].values,
                  xmax=parameter_df['recovered_hpd_upper'].values - parameter_df['generating'].values,
                  color=color)

        # Labels
        ax.set_xlabel(r'$\Delta$' + parameter_names[parameter])
        ax.set_ylabel('Participant')
        ax.set_yticks([])
        
        # Limits
        ax.set_xlim(*xlimits[parameter])

        # Panel Labels
        from string import ascii_lowercase
        for label, ax in zip(list(ascii_lowercase), axs.ravel()):
            ax.text(-0.1, 1, label, transform=ax.transAxes,
                    fontsize=12, fontweight='bold', va='top')
        
    fig.tight_layout()
    return fig, axs

In [13]:
plot_recovery_individual(glam_rec, true_parameters);
plt.savefig(join('examples', 'example_3', 'figures', 'ex3_fig2_deltaGenRec.png'), dpi=300)

In the example, all parameters could be recovered to a satisfying degree. Maximum a posteriori parameter estimates were close to the true generating parameters, on the level of the group (Fig. X, top row) and individual (Fig. X, bottom row).


# Conclusion

In this example, we demonstrated how to perform a basic parameter recovery for a given dataset. When successful, this increases confidence that the parameters can be identified with the given data set and that the obtained parameter estimates are the true generating parameters.
